In [ ]:
# =============================================================
# ✅ Load Official Multi-News Dataset (from Alex-Fabbri repo)
# =============================================================

!apt-get install git -y > /dev/null
!git clone https://github.com/Alex-Fabbri/Multi-News.git
!mkdir -p /content/multinews_data
!mv Multi-News/data/* /content/multinews_data/
!rm -rf Multi-News

import pandas as pd
from datasets import Dataset, DatasetDict

def load_split(src_path, tgt_path):
    """Load a Multi-News split from .src and .tgt text files"""
    with open(src_path, 'r', encoding='utf-8') as fsrc, open(tgt_path, 'r', encoding='utf-8') as ftgt:
        docs = [line.strip() for line in fsrc]
        sums = [line.strip() for line in ftgt]
    print(f"✅ Loaded {len(docs)} examples from {src_path}")
    return Dataset.from_pandas(pd.DataFrame({"document": docs, "summary": sums}))

# Build the dataset dict (train / validation / test)
dataset = DatasetDict({
    "train": load_split("/content/multinews_data/train.src", "/content/multinews_data/train.tgt"),
    "validation": load_split("/content/multinews_data/val.src", "/content/multinews_data/val.tgt"),
    "test": load_split("/content/multinews_data/test.src", "/content/multinews_data/test.tgt"),
})

print(dataset)


fatal: destination path 'Multi-News' already exists and is not an empty directory.
mv: cannot stat 'Multi-News/data/*': No such file or directory


FileNotFoundError: [Errno 2] No such file or directory: '/content/multinews_data/train.src'

In [ ]:
# =============================================================
# LED (Longformer Encoder–Decoder) Summarization - Multi-News
# ✅ Uses working mirror (knkarthick/multi_news_parquet)
# ✅ Automatically falls back to CNN/DailyMail if offline
# ✅ Saves progress to Google Drive with resume support
# =============================================================

!pip install transformers datasets rouge-score accelerate --quiet

import torch, gc, os, pandas as pd, sys
from transformers import AutoTokenizer, LEDForConditionalGeneration
from datasets import load_dataset

# =============================================================
# Mount Google Drive
# =============================================================
from google.colab import drive
drive.mount('/content/drive')

# =============================================================
# Config
# =============================================================
MODEL_NAME = "allenai/led-large-16384"
BATCH_SIZE = 1
CHUNK_SIZE = 50
LIMIT = 300                      # process fewer for speed
SPLIT = "validation"
SAVE_PATH = "/content/drive/MyDrive/summaries_led_multinews_val.csv"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("✅ Using device:", device)

# =============================================================
# Load model + tokenizer
# =============================================================
torch.cuda.empty_cache(); gc.collect()
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

try:
    model = LEDForConditionalGeneration.from_pretrained(
        MODEL_NAME,
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
        low_cpu_mem_usage=True
    ).to(device)
except RuntimeError:
    print("⚠️ GPU OOM → switching to CPU")
    model = LEDForConditionalGeneration.from_pretrained(MODEL_NAME).to("cpu")
    device = torch.device("cpu")

model.eval()

# =============================================================
# Load Multi-News (working mirror or fallback)
# =============================================================
print("📥 Loading Multi-News dataset (knkarthick/multi_news_parquet)...")

try:
    ds = load_dataset("knkarthick/multi_news_parquet")
    print("✅ Loaded Multi-News Parquet version.")
except Exception as e:
    print("⚠️ Could not load Multi-News. Using CNN/DailyMail fallback.")
    ds = load_dataset("cnn_dailymail", "3.0.0")

print(ds)

# Choose split safely
if SPLIT not in ds:
    SPLIT = "validation"

# Ensure correct field names
if "document" not in ds[SPLIT].column_names:
    if "article" in ds[SPLIT].column_names:
        articles = ds[SPLIT]["article"][:LIMIT]
        references = ds[SPLIT]["highlights"][:LIMIT]
    else:
        raise ValueError("Dataset does not contain expected text fields.")
else:
    articles = ds[SPLIT]["document"][:LIMIT]
    references = ds[SPLIT]["summary"][:LIMIT]

print(f"📚 Loaded {len(articles)} samples from split: {SPLIT}")

# =============================================================
# Summarization helper
# =============================================================
def summarize_batch(texts):
    inputs = tokenizer(
        texts,
        max_length=16384,
        truncation=True,
        padding=True,
        return_tensors="pt"
    ).to(device)

    with torch.no_grad():
        summary_ids = model.generate(
            **inputs,
            max_length=512,
            min_length=80,
            num_beams=2,
            early_stopping=True
        )

    outputs = tokenizer.batch_decode(summary_ids, skip_special_tokens=True)
    del inputs, summary_ids
    torch.cuda.empty_cache(); gc.collect()
    return outputs

# =============================================================
# Save progress to Drive
# =============================================================
def save_progress(articles, refs, summaries, save_path):
    df = pd.DataFrame({
        "document": articles,
        "reference": refs,
        "summary": summaries
    })
    header = not os.path.exists(save_path)
    df.to_csv(save_path, mode='a', header=header, index=False)

# =============================================================
# Resume support
# =============================================================
start_idx = 0
if os.path.exists(SAVE_PATH):
    df_prev = pd.read_csv(SAVE_PATH)
    start_idx = len(df_prev)
    print(f"🔁 Resuming from index {start_idx}")

# =============================================================
# Main summarization loop
# =============================================================
total_chunks = (len(articles) - start_idx + CHUNK_SIZE - 1) // CHUNK_SIZE
chunk_no = 1

for i in range(start_idx, len(articles), CHUNK_SIZE):
    end = min(i + CHUNK_SIZE, len(articles))
    print(f"\n🚀 Chunk {chunk_no}/{total_chunks} → Samples {i}–{end}")

    batch_articles = articles[i:end]
    batch_refs = references[i:end]
    all_summaries = []

    total_batches = (len(batch_articles) + BATCH_SIZE - 1) // BATCH_SIZE

    for j in range(0, len(batch_articles), BATCH_SIZE):
        sub_batch = batch_articles[j:j + BATCH_SIZE]
        batch_no = (j // BATCH_SIZE) + 1

        try:
            batch_summaries = summarize_batch(sub_batch)
        except torch.cuda.OutOfMemoryError:
            print("⚠️ OOM, retrying after clearing cache...")
            torch.cuda.empty_cache(); gc.collect()
            batch_summaries = summarize_batch(sub_batch)

        all_summaries.extend(batch_summaries)
        sys.stdout.write(f"\r[Batch {batch_no}/{total_batches}]")
        sys.stdout.flush()

    print()
    save_progress(batch_articles, batch_refs, all_summaries, SAVE_PATH)
    print(f"💾 Saved chunk {chunk_no}/{total_chunks} ({end}/{len(articles)} done)")

    chunk_no += 1
    torch.cuda.empty_cache(); gc.collect()

print("\n🎉 All Multi-News summaries complete! Saved to:", SAVE_PATH)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Using device: cuda
📥 Loading Multi-News dataset (knkarthick/multi_news_parquet)...
⚠️ Could not load Multi-News. Using CNN/DailyMail fallback.


README.md: 0.00B [00:00, ?B/s]

3.0.0/train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
# =============================================================
# LED (Longformer Encoder–Decoder) Summarization - Custom Drive ZIP
# ✅ Loads val.src.cleaned and val.tgt from ZIP in Google Drive
# ✅ Works even if ZIP already extracted
# ✅ Saves summaries to Google Drive with resume support
# =============================================================

!pip install transformers datasets rouge-score accelerate --quiet

import torch, gc, os, pandas as pd, sys, zipfile, glob
from transformers import AutoTokenizer, LEDForConditionalGeneration

# =============================================================
# Mount Google Drive
# =============================================================
from google.colab import drive
drive.mount('/content/drive')

# =============================================================
# Config
# =============================================================
MODEL_NAME = "allenai/led-large-16384"
BATCH_SIZE = 1
CHUNK_SIZE = 50
LIMIT = 500                        # Process fewer for faster testing
SPLIT = "val"                      # 👈 Focus only on validation split
ZIP_PATH = "/content/drive/MyDrive/multi_news.zip"
EXTRACT_DIR = "/content/data_multinews"
SAVE_PATH = f"/content/drive/MyDrive/summaries_led_multinews_{SPLIT}.csv"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("✅ Using device:", device)

# =============================================================
# Load model + tokenizer
# =============================================================
torch.cuda.empty_cache(); gc.collect()
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

try:
    model = LEDForConditionalGeneration.from_pretrained(
        MODEL_NAME,
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
        low_cpu_mem_usage=True
    ).to(device)
except RuntimeError:
    print("⚠️ GPU OOM → switching to CPU")
    model = LEDForConditionalGeneration.from_pretrained(MODEL_NAME).to("cpu")
    device = torch.device("cpu")

model.eval()

# =============================================================
# Load dataset from ZIP (val.src.cleaned + val.tgt)
# =============================================================
if not os.path.exists(EXTRACT_DIR):
    print("📦 Extracting dataset from Drive ZIP...")
    with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
        zip_ref.extractall(EXTRACT_DIR)
    print("✅ Extraction complete!")
else:
    print("♻️ Using already extracted data folder.")

# Find val.src.cleaned and val.tgt files
src_file, tgt_file = None, None
for root, dirs, files in os.walk(EXTRACT_DIR):
    for f in files:
        if f.startswith(SPLIT) and "src" in f and f.endswith(".cleaned"):
            src_file = os.path.join(root, f)
        elif f.startswith(SPLIT) and "tgt" in f:
            tgt_file = os.path.join(root, f)

if not src_file or not tgt_file:
    raise ValueError(f"❌ Could not find val.src.cleaned or val.tgt in extracted ZIP!")

print(f"✅ Found source file: {src_file}")
print(f"✅ Found target file: {tgt_file}")

# Load lines from files
with open(src_file, 'r', encoding='utf-8') as f:
    articles = [line.strip() for line in f.readlines()]

with open(tgt_file, 'r', encoding='utf-8') as f:
    references = [line.strip() for line in f.readlines()]

# Clip to same length + limit
min_len = min(len(articles), len(references))
articles = articles[:min_len][:LIMIT]
references = references[:min_len][:LIMIT]

print(f"📚 Loaded {len(articles)} validation samples.")

# =============================================================
# Summarization helper
# =============================================================
def summarize_batch(texts):
    inputs = tokenizer(
        texts,
        max_length=16384,
        truncation=True,
        padding=True,
        return_tensors="pt"
    ).to(device)

    with torch.no_grad():
        summary_ids = model.generate(
            **inputs,
            max_length=512,
            min_length=80,
            num_beams=2,
            early_stopping=True
        )

    outputs = tokenizer.batch_decode(summary_ids, skip_special_tokens=True)
    del inputs, summary_ids
    torch.cuda.empty_cache(); gc.collect()
    return outputs

# =============================================================
# Save progress
# =============================================================
def save_progress(articles, refs, summaries, save_path):
    df_out = pd.DataFrame({
        "document": articles,
        "reference": refs,
        "summary": summaries
    })
    header = not os.path.exists(save_path)
    df_out.to_csv(save_path, mode='a', header=header, index=False)

# =============================================================
# Resume support
# =============================================================
start_idx = 0
if os.path.exists(SAVE_PATH):
    df_prev = pd.read_csv(SAVE_PATH)
    start_idx = len(df_prev)
    print(f"🔁 Resuming from index {start_idx}")

# =============================================================
# Main summarization loop
# =============================================================
total_chunks = (len(articles) - start_idx + CHUNK_SIZE - 1) // CHUNK_SIZE
chunk_no = 1

for i in range(start_idx, len(articles), CHUNK_SIZE):
    end = min(i + CHUNK_SIZE, len(articles))
    print(f"\n🚀 Chunk {chunk_no}/{total_chunks} → Samples {i}–{end}")

    batch_articles = articles[i:end]
    batch_refs = references[i:end]
    all_summaries = []

    total_batches = (len(batch_articles) + BATCH_SIZE - 1) // BATCH_SIZE

    for j in range(0, len(batch_articles), BATCH_SIZE):
        sub_batch = batch_articles[j:j + BATCH_SIZE]
        batch_no = (j // BATCH_SIZE) + 1

        try:
            batch_summaries = summarize_batch(sub_batch)
        except torch.cuda.OutOfMemoryError:
            print("⚠️ OOM, retrying after clearing cache...")
            torch.cuda.empty_cache(); gc.collect()
            batch_summaries = summarize_batch(sub_batch)

        all_summaries.extend(batch_summaries)
        sys.stdout.write(f"\r[Batch {batch_no}/{total_batches}]")
        sys.stdout.flush()

    print()
    save_progress(batch_articles, batch_refs, all_summaries, SAVE_PATH)
    print(f"💾 Saved chunk {chunk_no}/{total_chunks} ({end}/{len(articles)} done)")

    chunk_no += 1
    torch.cuda.empty_cache(); gc.collect()

print(f"\n🎉 Validation summarization complete! Saved to: {SAVE_PATH}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Using device: cuda
♻️ Using already extracted data folder.
✅ Found source file: /content/data_multinews/multi_news/val.src.cleaned
✅ Found target file: /content/data_multinews/multi_news/val.tgt
📚 Loaded 500 validation samples.

🚀 Chunk 1/10 → Samples 0–50
[Batch 3/50]

Input ids are automatically padded from 980 to 1024 to be a multiple of `config.attention_window`: 1024


[Batch 4/50]

Input ids are automatically padded from 8479 to 9216 to be a multiple of `config.attention_window`: 1024


[Batch 5/50]

...

[Batch 48/50]

Input ids are automatically padded from 5984 to 6144 to be a multiple of `config.attention_window`: 1024


[Batch 49/50]

Input ids are automatically padded from 1203 to 2048 to be a multiple of `config.attention_window`: 1024


[Batch 50/50]
💾 Saved chunk 10/10 (500/500 done)

🎉 Validation summarization complete! Saved to: /content/drive/MyDrive/summaries_led_multinews_val.csv


In [ ]:
# =============================================================
# LED (Longformer Encoder–Decoder) Summarization - Custom Drive ZIP
# ✅ Loads val.src.cleaned and val.tgt from ZIP in Google Drive
# ✅ Works even if ZIP already extracted
# ✅ Saves summaries to Google Drive with resume support
# =============================================================

!pip install transformers datasets rouge-score accelerate --quiet

import torch, gc, os, pandas as pd, sys, zipfile, glob
from transformers import AutoTokenizer, LEDForConditionalGeneration

# =============================================================
# Mount Google Drive
# =============================================================
from google.colab import drive
drive.mount('/content/drive')

# =============================================================
# Config
# =============================================================
MODEL_NAME = "allenai/led-large-16384"
BATCH_SIZE = 1
CHUNK_SIZE = 50
LIMIT = 1000                        # Process fewer for faster testing
SPLIT = "val"                      # 👈 Focus only on validation split
ZIP_PATH = "/content/drive/MyDrive/multi_news.zip"
EXTRACT_DIR = "/content/data_multinews"
SAVE_PATH = f"/content/drive/MyDrive/summaries_led_multinews_{SPLIT}.csv"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("✅ Using device:", device)

# =============================================================
# Load model + tokenizer
# =============================================================
torch.cuda.empty_cache(); gc.collect()
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

try:
    model = LEDForConditionalGeneration.from_pretrained(
        MODEL_NAME,
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
        low_cpu_mem_usage=True
    ).to(device)
except RuntimeError:
    print("⚠️ GPU OOM → switching to CPU")
    model = LEDForConditionalGeneration.from_pretrained(MODEL_NAME).to("cpu")
    device = torch.device("cpu")

model.eval()

# =============================================================
# Load dataset from ZIP (val.src.cleaned + val.tgt)
# =============================================================
if not os.path.exists(EXTRACT_DIR):
    print("📦 Extracting dataset from Drive ZIP...")
    with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
        zip_ref.extractall(EXTRACT_DIR)
    print("✅ Extraction complete!")
else:
    print("♻️ Using already extracted data folder.")

# Find val.src.cleaned and val.tgt files
src_file, tgt_file = None, None
for root, dirs, files in os.walk(EXTRACT_DIR):
    for f in files:
        if f.startswith(SPLIT) and "src" in f and f.endswith(".cleaned"):
            src_file = os.path.join(root, f)
        elif f.startswith(SPLIT) and "tgt" in f:
            tgt_file = os.path.join(root, f)

if not src_file or not tgt_file:
    raise ValueError(f"❌ Could not find val.src.cleaned or val.tgt in extracted ZIP!")

print(f"✅ Found source file: {src_file}")
print(f"✅ Found target file: {tgt_file}")

# Load lines from files
with open(src_file, 'r', encoding='utf-8') as f:
    articles = [line.strip() for line in f.readlines()]

with open(tgt_file, 'r', encoding='utf-8') as f:
    references = [line.strip() for line in f.readlines()]

# Clip to same length + limit
min_len = min(len(articles), len(references))
articles = articles[:min_len][:LIMIT]
references = references[:min_len][:LIMIT]

print(f"📚 Loaded {len(articles)} validation samples.")

# =============================================================
# Summarization helper
# =============================================================
def summarize_batch(texts):
    inputs = tokenizer(
        texts,
        max_length=16384,
        truncation=True,
        padding=True,
        return_tensors="pt"
    ).to(device)

    with torch.no_grad():
        summary_ids = model.generate(
            **inputs,
            max_length=512,
            min_length=80,
            num_beams=2,
            early_stopping=True
        )

    outputs = tokenizer.batch_decode(summary_ids, skip_special_tokens=True)
    del inputs, summary_ids
    torch.cuda.empty_cache(); gc.collect()
    return outputs

# =============================================================
# Save progress
# =============================================================
def save_progress(articles, refs, summaries, save_path):
    df_out = pd.DataFrame({
        "document": articles,
        "reference": refs,
        "summary": summaries
    })
    header = not os.path.exists(save_path)
    df_out.to_csv(save_path, mode='a', header=header, index=False)

# =============================================================
# Resume support
# =============================================================
start_idx = 0
if os.path.exists(SAVE_PATH):
    df_prev = pd.read_csv(SAVE_PATH)
    start_idx = len(df_prev)
    print(f"🔁 Resuming from index {start_idx}")

# =============================================================
# Main summarization loop
# =============================================================
total_chunks = (len(articles) - start_idx + CHUNK_SIZE - 1) // CHUNK_SIZE
chunk_no = 1

for i in range(start_idx, len(articles), CHUNK_SIZE):
    end = min(i + CHUNK_SIZE, len(articles))
    print(f"\n🚀 Chunk {chunk_no}/{total_chunks} → Samples {i}–{end}")

    batch_articles = articles[i:end]
    batch_refs = references[i:end]
    all_summaries = []

    total_batches = (len(batch_articles) + BATCH_SIZE - 1) // BATCH_SIZE

    for j in range(0, len(batch_articles), BATCH_SIZE):
        sub_batch = batch_articles[j:j + BATCH_SIZE]
        batch_no = (j // BATCH_SIZE) + 1

        try:
            batch_summaries = summarize_batch(sub_batch)
        except torch.cuda.OutOfMemoryError:
            print("⚠️ OOM, retrying after clearing cache...")
            torch.cuda.empty_cache(); gc.collect()
            batch_summaries = summarize_batch(sub_batch)

        all_summaries.extend(batch_summaries)
        sys.stdout.write(f"\r[Batch {batch_no}/{total_batches}]")
        sys.stdout.flush()

    print()
    save_progress(batch_articles, batch_refs, all_summaries, SAVE_PATH)
    print(f"💾 Saved chunk {chunk_no}/{total_chunks} ({end}/{len(articles)} done)")

    chunk_no += 1
    torch.cuda.empty_cache(); gc.collect()

print(f"\n🎉 Validation summarization complete! Saved to: {SAVE_PATH}")


  Preparing metadata (setup.py) ... done
Mounted at /content/drive
✅ Using device: cuda


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

[Batch 48/50]

Input ids are automatically padded from 4486 to 5120 to be a multiple of `config.attention_window`: 1024


[Batch 49/50]

Input ids are automatically padded from 1034 to 2048 to be a multiple of `config.attention_window`: 1024


[Batch 50/50]
💾 Saved chunk 10/10 (1000/1000 done)

🎉 Validation summarization complete! Saved to: /content/drive/MyDrive/summaries_led_multinews_val.csv


In [ ]:
# =============================================================
# LED (Longformer Encoder–Decoder) Summarization - Custom Drive ZIP
# ✅ Loads test.src.cleaned and test.tgt from ZIP in Google Drive
# ✅ Works even if ZIP already extracted
# ✅ Inline progress updates (same-line batch display)
# ✅ Saves summaries to Google Drive with resume support
# =============================================================

!pip install transformers datasets rouge-score accelerate --quiet

import torch, gc, os, pandas as pd, sys, zipfile
from transformers import AutoTokenizer, LEDForConditionalGeneration
import warnings, transformers

# =============================================================
# Mount Google Drive
# =============================================================
from google.colab import drive
drive.mount('/content/drive')

# =============================================================
# Config
# =============================================================
MODEL_NAME = "allenai/led-large-16384"
BATCH_SIZE = 1
CHUNK_SIZE = 50
LIMIT = 500                        # Adjust for speed / testing
SPLIT = "test"                     # 👈 Focus only on TEST split
ZIP_PATH = "/content/drive/MyDrive/multi_news.zip"
EXTRACT_DIR = "/content/data_multinews"
SAVE_PATH = f"/content/drive/MyDrive/summaries_led_multinews_{SPLIT}.csv"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("✅ Using device:", device)

# =============================================================
# Load model + tokenizer
# =============================================================
torch.cuda.empty_cache(); gc.collect()
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

try:
    model = LEDForConditionalGeneration.from_pretrained(
        MODEL_NAME,
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
        low_cpu_mem_usage=True
    ).to(device)
except RuntimeError:
    print("⚠️ GPU OOM → switching to CPU")
    model = LEDForConditionalGeneration.from_pretrained(MODEL_NAME).to("cpu")
    device = torch.device("cpu")

model.eval()

# =============================================================
# Load dataset from ZIP (test.src.cleaned + test.tgt)
# =============================================================
if not os.path.exists(EXTRACT_DIR):
    print("📦 Extracting dataset from Drive ZIP...")
    with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
        zip_ref.extractall(EXTRACT_DIR)
    print("✅ Extraction complete!")
else:
    print("♻️ Using already extracted data folder.")

# Find test.src.cleaned and test.tgt files
src_file, tgt_file = None, None
for root, dirs, files in os.walk(EXTRACT_DIR):
    for f in files:
        if f.startswith(SPLIT) and "src" in f and f.endswith(".cleaned"):
            src_file = os.path.join(root, f)
        elif f.startswith(SPLIT) and "tgt" in f:
            tgt_file = os.path.join(root, f)

if not src_file or not tgt_file:
    raise ValueError(f"❌ Could not find {SPLIT}.src.cleaned or {SPLIT}.tgt in extracted ZIP!")

print(f"✅ Found source file: {src_file}")
print(f"✅ Found target file: {tgt_file}")

# Load lines from files
with open(src_file, 'r', encoding='utf-8') as f:
    articles = [line.strip() for line in f.readlines()]

with open(tgt_file, 'r', encoding='utf-8') as f:
    references = [line.strip() for line in f.readlines()]

# Clip to same length + limit
min_len = min(len(articles), len(references))
articles = articles[:min_len][:LIMIT]
references = references[:min_len][:LIMIT]

print(f"📚 Loaded {len(articles)} test samples.")

# =============================================================
# Summarization helper
# =============================================================
def summarize_batch(texts):
    inputs = tokenizer(
        texts,
        max_length=16384,
        truncation=True,
        padding=True,
        return_tensors="pt"
    ).to(device)

    with torch.no_grad():
        summary_ids = model.generate(
            **inputs,
            max_length=512,
            min_length=80,
            num_beams=2,
            early_stopping=True
        )

    outputs = tokenizer.batch_decode(summary_ids, skip_special_tokens=True)
    del inputs, summary_ids
    torch.cuda.empty_cache(); gc.collect()
    return outputs

# =============================================================
# Save progress
# =============================================================
def save_progress(articles, refs, summaries, save_path):
    df_out = pd.DataFrame({
        "document": articles,
        "reference": refs,
        "summary": summaries
    })
    header = not os.path.exists(save_path)
    df_out.to_csv(save_path, mode='a', header=header, index=False)

# =============================================================
# Resume support
# =============================================================
start_idx = 0
if os.path.exists(SAVE_PATH):
    df_prev = pd.read_csv(SAVE_PATH)
    start_idx = len(df_prev)
    print(f"🔁 Resuming from index {start_idx}")

# =============================================================
# Clean up noisy LED padding warnings
# =============================================================
warnings.filterwarnings("ignore", message="Input ids are automatically padded")
transformers.logging.set_verbosity_error()

# =============================================================
# Main summarization loop (dynamic same-line batch updates)
# =============================================================
total_chunks = (len(articles) - start_idx + CHUNK_SIZE - 1) // CHUNK_SIZE
chunk_no = 1

for i in range(start_idx, len(articles), CHUNK_SIZE):
    end = min(i + CHUNK_SIZE, len(articles))
    print(f"\n🚀 Chunk {chunk_no}/{total_chunks} → Samples {i}–{end}")

    batch_articles = articles[i:end]
    batch_refs = references[i:end]
    all_summaries = []

    total_batches = (len(batch_articles) + BATCH_SIZE - 1) // BATCH_SIZE

    for j in range(0, len(batch_articles), BATCH_SIZE):
        sub_batch = batch_articles[j:j + BATCH_SIZE]
        batch_no = (j // BATCH_SIZE) + 1

        try:
            batch_summaries = summarize_batch(sub_batch)
        except torch.cuda.OutOfMemoryError:
            torch.cuda.empty_cache(); gc.collect()
            batch_summaries = summarize_batch(sub_batch)

        all_summaries.extend(batch_summaries)

        # ✅ overwrite the same line (no newlines for each batch)
        sys.stdout.write(f"\r[Chunk {chunk_no}/{total_chunks}] [Batch {batch_no}/{total_batches}]")
        sys.stdout.flush()

    # move to new line after chunk finishes
    print()
    save_progress(batch_articles, batch_refs, all_summaries, SAVE_PATH)
    print(f"💾 Saved chunk {chunk_no}/{total_chunks} ({end}/{len(articles)} done)")

    chunk_no += 1
    torch.cuda.empty_cache(); gc.collect()

print(f"\n🎉 Test summarization complete! Saved to: {SAVE_PATH}")


  Preparing metadata (setup.py) ... done
Mounted at /content/drive
✅ Using device: cuda


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

`torch_dtype` is deprecated! Use `dtype` instead!


pytorch_model.bin:   0%|          | 0.00/1.84G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.84G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

📦 Extracting dataset from Drive ZIP...
✅ Extraction complete!
✅ Found source file: /content/data_multinews/multi_news/test.src.cleaned
✅ Found target file: /content/data_multinews/multi_news/test.tgt
📚 Loaded 500 test samples.

🚀 Chunk 1/10 → Samples 0–50
[Chunk 1/10] [Batch 50/50]
💾 Saved chunk 1/10 (50/500 done)

🚀 Chunk 2/10 → Samples 50–100
[Chunk 2/10] [Batch 50/50]
💾 Saved chunk 2/10 (100/500 done)

🚀 Chunk 3/10 → Samples 100–150
[Chunk 3/10] [Batch 50/50]
💾 Saved chunk 3/10 (150/500 done)

🚀 Chunk 4/10 → Samples 150–200
[Chunk 4/10] [Batch 50/50]
💾 Saved chunk 4/10 (200/500 done)

🚀 Chunk 5/10 → Samples 200–250
[Chunk 5/10] [Batch 50/50]
💾 Saved chunk 5/10 (250/500 done)

🚀 Chunk 6/10 → Samples 250–300
[Chunk 6/10] [Batch 50/50]
💾 Saved chunk 6/10 (300/500 done)

🚀 Chunk 7/10 → Samples 300–350
[Chunk 7/10] [Batch 50/50]
💾 Saved chunk 7/10 (350/500 done)

🚀 Chunk 8/10 → Samples 350–400
[Chunk 8/10] [Batch 50/50]
💾 Saved chunk 8/10 (400/500 done)

🚀 Chunk 9/10 → Samples 400–450
[C

In [ ]:
# =============================================================
# LED (Longformer Encoder–Decoder) Summarization - Custom Drive ZIP
# ✅ Loads test.src.cleaned and test.tgt from ZIP in Google Drive
# ✅ Works even if ZIP already extracted
# ✅ Inline progress updates (same-line batch display)
# ✅ Saves summaries to Google Drive with resume support
# =============================================================

!pip install transformers datasets rouge-score accelerate --quiet

import torch, gc, os, pandas as pd, sys, zipfile
from transformers import AutoTokenizer, LEDForConditionalGeneration
import warnings, transformers

# =============================================================
# Mount Google Drive
# =============================================================
from google.colab import drive
drive.mount('/content/drive')

# =============================================================
# Config
# =============================================================
MODEL_NAME = "allenai/led-large-16384"
BATCH_SIZE = 1
CHUNK_SIZE = 50
LIMIT = 1000                        # Adjust for speed / testing
SPLIT = "test"                     # 👈 Focus only on TEST split
ZIP_PATH = "/content/drive/MyDrive/multi_news.zip"
EXTRACT_DIR = "/content/data_multinews"
SAVE_PATH = f"/content/drive/MyDrive/summaries_led_multinews_{SPLIT}.csv"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("✅ Using device:", device)

# =============================================================
# Load model + tokenizer
# =============================================================
torch.cuda.empty_cache(); gc.collect()
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

try:
    model = LEDForConditionalGeneration.from_pretrained(
        MODEL_NAME,
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
        low_cpu_mem_usage=True
    ).to(device)
except RuntimeError:
    print("⚠️ GPU OOM → switching to CPU")
    model = LEDForConditionalGeneration.from_pretrained(MODEL_NAME).to("cpu")
    device = torch.device("cpu")

model.eval()

# =============================================================
# Load dataset from ZIP (test.src.cleaned + test.tgt)
# =============================================================
if not os.path.exists(EXTRACT_DIR):
    print("📦 Extracting dataset from Drive ZIP...")
    with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
        zip_ref.extractall(EXTRACT_DIR)
    print("✅ Extraction complete!")
else:
    print("♻️ Using already extracted data folder.")

# Find test.src.cleaned and test.tgt files
src_file, tgt_file = None, None
for root, dirs, files in os.walk(EXTRACT_DIR):
    for f in files:
        if f.startswith(SPLIT) and "src" in f and f.endswith(".cleaned"):
            src_file = os.path.join(root, f)
        elif f.startswith(SPLIT) and "tgt" in f:
            tgt_file = os.path.join(root, f)

if not src_file or not tgt_file:
    raise ValueError(f"❌ Could not find {SPLIT}.src.cleaned or {SPLIT}.tgt in extracted ZIP!")

print(f"✅ Found source file: {src_file}")
print(f"✅ Found target file: {tgt_file}")

# Load lines from files
with open(src_file, 'r', encoding='utf-8') as f:
    articles = [line.strip() for line in f.readlines()]

with open(tgt_file, 'r', encoding='utf-8') as f:
    references = [line.strip() for line in f.readlines()]

# Clip to same length + limit
min_len = min(len(articles), len(references))
articles = articles[:min_len][:LIMIT]
references = references[:min_len][:LIMIT]

print(f"📚 Loaded {len(articles)} test samples.")

# =============================================================
# Summarization helper
# =============================================================
def summarize_batch(texts):
    inputs = tokenizer(
        texts,
        max_length=16384,
        truncation=True,
        padding=True,
        return_tensors="pt"
    ).to(device)

    with torch.no_grad():
        summary_ids = model.generate(
            **inputs,
            max_length=512,
            min_length=80,
            num_beams=2,
            early_stopping=True
        )

    outputs = tokenizer.batch_decode(summary_ids, skip_special_tokens=True)
    del inputs, summary_ids
    torch.cuda.empty_cache(); gc.collect()
    return outputs

# =============================================================
# Save progress
# =============================================================
def save_progress(articles, refs, summaries, save_path):
    df_out = pd.DataFrame({
        "document": articles,
        "reference": refs,
        "summary": summaries
    })
    header = not os.path.exists(save_path)
    df_out.to_csv(save_path, mode='a', header=header, index=False)

# =============================================================
# Resume support
# =============================================================
start_idx = 0
if os.path.exists(SAVE_PATH):
    df_prev = pd.read_csv(SAVE_PATH)
    start_idx = len(df_prev)
    print(f"🔁 Resuming from index {start_idx}")

# =============================================================
# Clean up noisy LED padding warnings
# =============================================================
warnings.filterwarnings("ignore", message="Input ids are automatically padded")
transformers.logging.set_verbosity_error()

# =============================================================
# Main summarization loop (dynamic same-line batch updates)
# =============================================================
total_chunks = (len(articles) - start_idx + CHUNK_SIZE - 1) // CHUNK_SIZE
chunk_no = 1

for i in range(start_idx, len(articles), CHUNK_SIZE):
    end = min(i + CHUNK_SIZE, len(articles))
    print(f"\n🚀 Chunk {chunk_no}/{total_chunks} → Samples {i}–{end}")

    batch_articles = articles[i:end]
    batch_refs = references[i:end]
    all_summaries = []

    total_batches = (len(batch_articles) + BATCH_SIZE - 1) // BATCH_SIZE

    for j in range(0, len(batch_articles), BATCH_SIZE):
        sub_batch = batch_articles[j:j + BATCH_SIZE]
        batch_no = (j // BATCH_SIZE) + 1

        try:
            batch_summaries = summarize_batch(sub_batch)
        except torch.cuda.OutOfMemoryError:
            torch.cuda.empty_cache(); gc.collect()
            batch_summaries = summarize_batch(sub_batch)

        all_summaries.extend(batch_summaries)

        # ✅ overwrite the same line (no newlines for each batch)
        sys.stdout.write(f"\r[Chunk {chunk_no}/{total_chunks}] [Batch {batch_no}/{total_batches}]")
        sys.stdout.flush()

    # move to new line after chunk finishes
    print()
    save_progress(batch_articles, batch_refs, all_summaries, SAVE_PATH)
    print(f"💾 Saved chunk {chunk_no}/{total_chunks} ({end}/{len(articles)} done)")

    chunk_no += 1
    torch.cuda.empty_cache(); gc.collect()

print(f"\n🎉 Test summarization complete! Saved to: {SAVE_PATH}")


Mounted at /content/drive
✅ Using device: cuda


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

`torch_dtype` is deprecated! Use `dtype` instead!


pytorch_model.bin:   0%|          | 0.00/1.84G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.84G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

📦 Extracting dataset from Drive ZIP...
✅ Extraction complete!
✅ Found source file: /content/data_multinews/multi_news/test.src.cleaned
✅ Found target file: /content/data_multinews/multi_news/test.tgt
📚 Loaded 1000 test samples.
🔁 Resuming from index 600

🚀 Chunk 1/8 → Samples 600–650
[Chunk 1/8] [Batch 50/50]
💾 Saved chunk 1/8 (650/1000 done)

🚀 Chunk 2/8 → Samples 650–700
[Chunk 2/8] [Batch 50/50]
💾 Saved chunk 2/8 (700/1000 done)

🚀 Chunk 3/8 → Samples 700–750
[Chunk 3/8] [Batch 50/50]
💾 Saved chunk 3/8 (750/1000 done)

🚀 Chunk 4/8 → Samples 750–800
[Chunk 4/8] [Batch 50/50]
💾 Saved chunk 4/8 (800/1000 done)

🚀 Chunk 5/8 → Samples 800–850
[Chunk 5/8] [Batch 50/50]
💾 Saved chunk 5/8 (850/1000 done)

🚀 Chunk 6/8 → Samples 850–900
[Chunk 6/8] [Batch 50/50]
💾 Saved chunk 6/8 (900/1000 done)

🚀 Chunk 7/8 → Samples 900–950
[Chunk 7/8] [Batch 50/50]
💾 Saved chunk 7/8 (950/1000 done)

🚀 Chunk 8/8 → Samples 950–1000
[Chunk 8/8] [Batch 50/50]
💾 Saved chunk 8/8 (1000/1000 done)

🎉 Test summariz

In [ ]:
# =============================================================
# ROUGE Evaluation (Fixed for 'evaluate' library)
# =============================================================

!pip install evaluate --quiet

import evaluate
import pandas as pd
import numpy as np

# Load your saved CSV (replace with your actual path if needed)
df = pd.read_csv(SAVE_PATH)
df = df.dropna(subset=["reference", "summary"])

# Initialize ROUGE metric
rouge = evaluate.load("rouge")

# Compute scores
results = rouge.compute(
    predictions=df["summary"].tolist(),
    references=df["reference"].tolist(),
    use_stemmer=True
)

# Extract F1 values (evaluate returns floats directly, not objects with .mid)
r1 = results["rouge1"]
r2 = results["rouge2"]
rL = results["rougeL"]

# Compute average precision across all three
avg_precision = np.mean([r1, r2, rL])

# Show neatly
def pct(x): return round(x * 100, 2)
print("\n================ ROUGE RESULTS ================")
print(f"ROUGE-1: {pct(r1)}")
print(f"ROUGE-2: {pct(r2)}")
print(f"ROUGE-L: {pct(rL)}")
print(f"Average Precision: {pct(avg_precision)}")
print("===============================================")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.9 MB/s eta 0:00:00



================ ROUGE RESULTS ================
ROUGE-1: 17.14
ROUGE-2: 4.85
ROUGE-L: 9.62
Average Precision: 10.54


In [ ]:
# =============================================================
# 🔍 ROUGE Evaluation for Validation Split (Updated for evaluate)
# =============================================================

!pip install evaluate --quiet

import evaluate
import pandas as pd
import numpy as np
import os

# =============================================================
# Paths (adjust for your validation run)
# =============================================================
SPLIT = "val"  # 👈 change to "test" if needed
SAVE_PATH = f"/content/drive/MyDrive/summaries_led_multinews_{SPLIT}.csv"
EXTRACT_DIR = "/content/data_multinews"

# =============================================================
# Load generated summaries (your model outputs)
# =============================================================
if not os.path.exists(SAVE_PATH):
    raise FileNotFoundError(f"❌ No saved summaries found at {SAVE_PATH}")

df_gen = pd.read_csv(SAVE_PATH)
df_gen = df_gen.dropna(subset=["summary"])
print(f"📄 Loaded {len(df_gen)} generated {SPLIT} summaries.")

# =============================================================
# Load ground-truth validation references (e.g., val.tgt)
# =============================================================
val_tgt = None
for root, dirs, files in os.walk(EXTRACT_DIR):
    for f in files:
        if f.startswith(SPLIT) and "tgt" in f:
            val_tgt = os.path.join(root, f)

if not val_tgt:
    raise ValueError(f"❌ Could not find {SPLIT}.tgt file in dataset folder!")

with open(val_tgt, 'r', encoding='utf-8') as f:
    references = [line.strip() for line in f.readlines()]

# Ensure equal number of samples
min_len = min(len(df_gen), len(references))
preds = df_gen["summary"].tolist()[:min_len]
refs = references[:min_len]

# =============================================================
# Compute ROUGE
# =============================================================
rouge = evaluate.load("rouge")

results = rouge.compute(
    predictions=preds,
    references=refs,
    use_stemmer=True
)

# Extract ROUGE scores (already F1-based)
r1 = results["rouge1"]
r2 = results["rouge2"]
rL = results["rougeL"]

# Average precision
avg_precision = np.mean([r1, r2, rL])

# =============================================================
# Display
# =============================================================
def pct(x): return round(x * 100, 2)
print("\n================ VAL ROUGE RESULTS ================")
print(f"ROUGE-1: {pct(r1)}")
print(f"ROUGE-2: {pct(r2)}")
print(f"ROUGE-L: {pct(rL)}")
print(f"Average Precision: {pct(avg_precision)}")
print("===================================================")


📄 Loaded 1000 generated val summaries.

================ VAL ROUGE RESULTS ================
ROUGE-1: 16.77
ROUGE-2: 4.56
ROUGE-L: 9.5
Average Precision: 10.27


In [ ]:
# =============================================================
# LED (Longformer Encoder–Decoder) Summarization - Custom Drive ZIP
# ✅ Loads test.src.cleaned and test.tgt from ZIP in Google Drive
# ✅ Works even if ZIP already extracted
# ✅ Inline progress updates (same-line batch display)
# ✅ Saves summaries to Google Drive with resume support
# =============================================================

!pip install transformers datasets rouge-score accelerate --quiet

import torch, gc, os, pandas as pd, sys, zipfile
from transformers import AutoTokenizer, LEDForConditionalGeneration
import warnings, transformers

# =============================================================
# Mount Google Drive
# =============================================================
from google.colab import drive
drive.mount('/content/drive')

# =============================================================
# Config
# =============================================================
MODEL_NAME = "allenai/led-large-16384"
BATCH_SIZE = 1
CHUNK_SIZE = 50
LIMIT = 1000                        # Adjust for speed / testing
SPLIT = "test"                     # 👈 Focus only on TEST split
ZIP_PATH = "/content/drive/MyDrive/multi_news.zip"
EXTRACT_DIR = "/content/data_multinews"
SAVE_PATH = f"/content/drive/MyDrive/summaries_led_multinews_{SPLIT}.csv"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("✅ Using device:", device)

# =============================================================
# Load model + tokenizer
# =============================================================
torch.cuda.empty_cache(); gc.collect()
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

try:
    model = LEDForConditionalGeneration.from_pretrained(
        MODEL_NAME,
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
        low_cpu_mem_usage=True
    ).to(device)
except RuntimeError:
    print("⚠️ GPU OOM → switching to CPU")
    model = LEDForConditionalGeneration.from_pretrained(MODEL_NAME).to("cpu")
    device = torch.device("cpu")

model.eval()

# =============================================================
# Load dataset from ZIP (test.src.cleaned + test.tgt)
# =============================================================
if not os.path.exists(EXTRACT_DIR):
    print("📦 Extracting dataset from Drive ZIP...")
    with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
        zip_ref.extractall(EXTRACT_DIR)
    print("✅ Extraction complete!")
else:
    print("♻️ Using already extracted data folder.")

# Find test.src.cleaned and test.tgt files
src_file, tgt_file = None, None
for root, dirs, files in os.walk(EXTRACT_DIR):
    for f in files:
        if f.startswith(SPLIT) and "src" in f and f.endswith(".cleaned"):
            src_file = os.path.join(root, f)
        elif f.startswith(SPLIT) and "tgt" in f:
            tgt_file = os.path.join(root, f)

if not src_file or not tgt_file:
    raise ValueError(f"❌ Could not find {SPLIT}.src.cleaned or {SPLIT}.tgt in extracted ZIP!")

print(f"✅ Found source file: {src_file}")
print(f"✅ Found target file: {tgt_file}")

# Load lines from files
with open(src_file, 'r', encoding='utf-8') as f:
    articles = [line.strip() for line in f.readlines()]

with open(tgt_file, 'r', encoding='utf-8') as f:
    references = [line.strip() for line in f.readlines()]

# Clip to same length + limit
min_len = min(len(articles), len(references))
articles = articles[:min_len][:LIMIT]
references = references[:min_len][:LIMIT]

print(f"📚 Loaded {len(articles)} test samples.")

# =============================================================
# Summarization helper
# =============================================================
def summarize_batch(texts):
    inputs = tokenizer(
        texts,
        max_length=16384,
        truncation=True,
        padding=True,
        return_tensors="pt"
    ).to(device)

    with torch.no_grad():
        summary_ids = model.generate(
            **inputs,
            max_length=512,
            min_length=80,
            num_beams=2,
            early_stopping=True
        )

    outputs = tokenizer.batch_decode(summary_ids, skip_special_tokens=True)
    del inputs, summary_ids
    torch.cuda.empty_cache(); gc.collect()
    return outputs

# =============================================================
# Save progress
# =============================================================
def save_progress(articles, refs, summaries, save_path):
    df_out = pd.DataFrame({
        "document": articles,
        "reference": refs,
        "summary": summaries
    })
    header = not os.path.exists(save_path)
    df_out.to_csv(save_path, mode='a', header=header, index=False)

# =============================================================
# Resume support
# =============================================================
start_idx = 0
if os.path.exists(SAVE_PATH):
    df_prev = pd.read_csv(SAVE_PATH)
    start_idx = len(df_prev)
    print(f"🔁 Resuming from index {start_idx}")

# =============================================================
# Clean up noisy LED padding warnings
# =============================================================
warnings.filterwarnings("ignore", message="Input ids are automatically padded")
transformers.logging.set_verbosity_error()

# =============================================================
# Main summarization loop (dynamic same-line batch updates)
# =============================================================
total_chunks = (len(articles) - start_idx + CHUNK_SIZE - 1) // CHUNK_SIZE
chunk_no = 1

for i in range(start_idx, len(articles), CHUNK_SIZE):
    end = min(i + CHUNK_SIZE, len(articles))
    print(f"\n🚀 Chunk {chunk_no}/{total_chunks} → Samples {i}–{end}")

    batch_articles = articles[i:end]
    batch_refs = references[i:end]
    all_summaries = []

    total_batches = (len(batch_articles) + BATCH_SIZE - 1) // BATCH_SIZE

    for j in range(0, len(batch_articles), BATCH_SIZE):
        sub_batch = batch_articles[j:j + BATCH_SIZE]
        batch_no = (j // BATCH_SIZE) + 1

        try:
            batch_summaries = summarize_batch(sub_batch)
        except torch.cuda.OutOfMemoryError:
            torch.cuda.empty_cache(); gc.collect()
            batch_summaries = summarize_batch(sub_batch)

        all_summaries.extend(batch_summaries)

        # ✅ overwrite the same line (no newlines for each batch)
        sys.stdout.write(f"\r[Chunk {chunk_no}/{total_chunks}] [Batch {batch_no}/{total_batches}]")
        sys.stdout.flush()

    # move to new line after chunk finishes
    print()
    save_progress(batch_articles, batch_refs, all_summaries, SAVE_PATH)
    print(f"💾 Saved chunk {chunk_no}/{total_chunks} ({end}/{len(articles)} done)")

    chunk_no += 1
    torch.cuda.empty_cache(); gc.collect()

print(f"\n🎉 Test summarization complete! Saved to: {SAVE_PATH}")


  Preparing metadata (setup.py) ... done
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Using device: cpu


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

`torch_dtype` is deprecated! Use `dtype` instead!


pytorch_model.bin:   0%|          | 0.00/1.84G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.84G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

📦 Extracting dataset from Drive ZIP...
✅ Extraction complete!
✅ Found source file: /content/data_multinews/multi_news/test.src.cleaned
✅ Found target file: /content/data_multinews/multi_news/test.tgt
📚 Loaded 1000 test samples.
🔁 Resuming from index 1000

🎉 Test summarization complete! Saved to: /content/drive/MyDrive/summaries_led_multinews_test.csv


In [ ]:
# =============================================================
# 🔍 Word-Level Average Precision for Validation & Test Splits
# =============================================================

!pip install nltk --quiet
import pandas as pd
import numpy as np
import os
from nltk.tokenize import word_tokenize
from tqdm import tqdm
import nltk

# -------------------------------------------------------------
# 🔧 Fix NLTK punkt issue
# -------------------------------------------------------------
nltk.download('punkt')
nltk.download('punkt_tab')

# =============================================================
# Function to compute Average Precision (word-overlap)
# =============================================================
def compute_avg_precision(split_name, base_dir, extract_dir):
    print(f"\n🔹 Evaluating split: {split_name.upper()}")

    # Load generated summaries
    save_path = os.path.join(base_dir, f"summaries_led_multinews_{split_name}.csv")
    if not os.path.exists(save_path):
        print(f"❌ Skipping — file not found: {save_path}")
        return None

    df_gen = pd.read_csv(save_path)
    df_gen = df_gen.dropna(subset=["summary"])
    print(f"📄 Loaded {len(df_gen)} generated summaries for {split_name}.")

    # Load references (.tgt file)
    tgt_path = None
    for root, _, files in os.walk(extract_dir):
        for f in files:
            if f.startswith(split_name) and "tgt" in f:
                tgt_path = os.path.join(root, f)
                break
    if not tgt_path:
        print(f"❌ No {split_name}.tgt file found.")
        return None

    with open(tgt_path, 'r', encoding='utf-8') as f:
        refs = [line.strip() for line in f.readlines()]

    # Ensure matching length
    min_len = min(len(df_gen), len(refs))
    preds = df_gen["summary"].tolist()[:min_len]
    refs = refs[:min_len]

    # Word-level precision
    precision_list = []
    for ref, summ in tqdm(zip(refs, preds), total=len(refs), desc=f"{split_name} precision"):
        ref_tokens = word_tokenize(ref.lower())
        summ_tokens = word_tokenize(summ.lower())

        if len(summ_tokens) == 0:
            precision_list.append(0)
            continue

        ref_set = set(ref_tokens)
        summ_set = set(summ_tokens)
        overlap = len(ref_set.intersection(summ_set))
        precision = overlap / len(summ_set)
        precision_list.append(precision)

    avg_precision = np.mean(precision_list)
    print(f"✅ {split_name.upper()} Average Precision (word overlap): {round(avg_precision * 100, 2)}%")
    return avg_precision


# =============================================================
# Run for both Validation and Test
# =============================================================
BASE_DIR = "/content/drive/MyDrive"
EXTRACT_DIR = "/content/data_multinews"

val_precision = compute_avg_precision("val", BASE_DIR, EXTRACT_DIR)
test_precision = compute_avg_precision("test", BASE_DIR, EXTRACT_DIR)

print("\n================ FINAL PRECISION RESULTS ================")
if val_precision is not None:
    print(f"VAL Average Precision: {round(val_precision * 100, 2)}%")
if test_precision is not None:
    print(f"TEST Average Precision: {round(test_precision * 100, 2)}%")
print("=========================================================")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!



🔹 Evaluating split: VAL
📄 Loaded 1000 generated summaries for val.


val precision: 100%|██████████| 1000/1000 [00:14<00:00, 71.30it/s]


✅ VAL Average Precision (word overlap): 40.67%

🔹 Evaluating split: TEST
📄 Loaded 1000 generated summaries for test.


test precision: 100%|██████████| 1000/1000 [00:16<00:00, 60.13it/s] 

✅ TEST Average Precision (word overlap): 39.97%

================ FINAL PRECISION RESULTS ================
VAL Average Precision: 40.67%
TEST Average Precision: 39.97%


In [ ]:
def summarize_batch(texts):
    inputs = tokenizer(
        texts,
        max_length=16384,
        truncation=True,
        padding=True,
        return_tensors="pt"
    )

    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)

    # ✅ LED-specific: global attention
    global_attention_mask = torch.zeros_like(attention_mask)
    global_attention_mask[:, 0] = 1   # CLS token gets global attention

    with torch.no_grad():
        summary_ids = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            global_attention_mask=global_attention_mask,
            max_length=512,
            min_length=80,
            num_beams=2,
            early_stopping=True
        )

    outputs = tokenizer.batch_decode(summary_ids, skip_special_tokens=True)
    del input_ids, attention_mask, global_attention_mask, summary_ids
    torch.cuda.empty_cache(); gc.collect()
    return outputs
